In [79]:
# import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
# %matplotlib inline

In [80]:
import datetime
import cPickle as pickle
import csv
import numpy as np
import random
import sys
from file_util import create_folder
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [81]:
name_dataset        = 'real_world_articles/old'
# name_dataset        = 'real_world_articles/new'
path_raw_data       = '../data/raw/' + name_dataset + '/'
path_processed_data = '../data/' + name_dataset + '/whole/'

In [82]:
create_folder(path_raw_data)
# create_folder(path_processed_data + '/train')
# create_folder(path_processed_data + '/dev')
create_folder(path_processed_data + '/rw-test')
# create_folder(path_processed_data + '/debug')

## read voca from dic

In [83]:
target_dic = '../data/news-19_paragraph_swap-random-1m/whole/dic_mincutN.txt'

list_voca = []
with open(target_dic, 'r') as f:
    list_voca = f.readlines()
    list_voca = [x.strip() for x in list_voca]

dic_voca = {}
for voca in list_voca:
    dic_voca[voca] = len(dic_voca)

len(dic_voca)

196779

## for data processing

In [84]:
import copy
dic_voca_lower = copy.deepcopy(dic_voca)

In [85]:
dic_voca_lower['<eos>'] = dic_voca_lower['<EOS>']
dic_voca_lower['<eop>'] = dic_voca_lower['<EOP>']

del dic_voca_lower['<EOS>']
del dic_voca_lower['<EOP>']

In [86]:
len(dic_voca_lower)

196779

In [87]:
print(dic_voca_lower[''], dic_voca_lower['<UNK>'], dic_voca_lower['<eos>'], dic_voca_lower['<eop>'])

(0, 1, 2, 3)


## stats

In [88]:
import csv
import sys
import numpy as np

data= []
with open(path_raw_data + 'rw-test.tsv', 'r') as f:
    data_csv = csv.reader(f, delimiter='\t',
                          quoting=csv.QUOTE_NONE)
    for row in data_csv:
        data.append(row)

In [89]:
def print_info(data):
    print("mean", np.average(data))
    print("std", np.std(data))
    print("max", np.max(data))
    print("95.xx coverage", np.average(data) +  2*np.std(data) )
    print("99.73 coverage", np.average(data) +  3*np.std(data) )
    print("99.95 coverage", np.average(data) +  3.5*np.std(data) )
    print("99.99 coverage", np.average(data) +  4*np.std(data) )

In [90]:
head = [x[1].strip() for x in data]
head_len = [len(x.split()) for x in head]
print('head_len')
print_info(head_len)

head_len
('mean', 11.716554324231693)
('std', 3.0670143983352007)
('max', 36)
('95.xx coverage', 17.850583120902094)
('99.73 coverage', 20.917597519237297)
('99.95 coverage', 22.451104718404896)
('99.99 coverage', 23.984611917572494)


In [91]:
body = [x[2].strip() for x in data]
body_len = [len(x.split()) for x in body ]
print('body_len')
print_info(body_len)

body_len
('mean', 536.8027647663658)
('std', 367.0733445702193)
('max', 10412)
('95.xx coverage', 1270.9494539068046)
('99.73 coverage', 1638.0227984770236)
('99.95 coverage', 1821.5594707621335)
('99.99 coverage', 2005.096143047243)


In [92]:
context_len = [len(x.split('<EOP>')) for x in body]
print('context_len')
print_info(context_len)

context_len
('mean', 12.37537986975894)
('std', 7.798870593796197)
('max', 49)
('95.xx coverage', 27.973121057351335)
('99.73 coverage', 35.77199165114753)
('99.95 coverage', 39.67142694804563)
('99.99 coverage', 43.570862244943726)


In [93]:
body_sentence = []
for sent in body:
    sent = sent.split('<EOP>')
    body_sentence.extend(sent)
body_len = [ len(x.split()) for x in body_sentence ]    
print('body_sentence')
print_info(body_len)

body_sentence
('mean', 42.45747527965311)
('std', 27.678158266390625)
('max', 7146)
('95.xx coverage', 97.81379181243436)
('99.73 coverage', 125.49195007882498)
('99.95 coverage', 139.3310292120203)
('99.99 coverage', 153.17010834521562)


# encode to numpy

In [94]:
def fit_length(data, max_len_t, max_len_b):
    data_t, data_b = data
    
    list_zeros = np.zeros(max_len_b, 'int32').tolist()
    fl_data_t = []
    for datum in data_t:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_t:
            fl_data_t.append( datum[:max_len_t] )
        else:
            fl_data_t.append( datum + list_zeros[:(max_len_t-_len)] )
            
    fl_data_b = []
    for datum in data_b:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_b:
            fl_data_b.append( datum[:max_len_b] )
        else:
            fl_data_b.append( datum + list_zeros[:(max_len_b-_len)] )
    
    np_data_t = np.asarray(fl_data_t, dtype='int32')
    np_data_b = np.asarray(fl_data_b, dtype='int32')
    
    data = [np_data_t, np_data_b]
    return data

In [95]:
csv_reader = csv.reader(open(path_raw_data + 'rw-test.tsv', 'r'),
                        delimiter='\t',
                        quoting=csv.QUOTE_NONE)

print datetime.datetime.now().isoformat()
ids = []
heads = []
bodys = []
labels = []
for n, row in enumerate(csv_reader):
    
#     if n <  3000000:
#         continue

#     if n >=  3000000:
#         continue
        
                
    if (n+1) % 10000 == 0: print n+1,
    
    ids.append(row[0])
    labels.append(0)
    
    head = []
    for tkn in row[1].lower().strip().split():
        if tkn in dic_voca_lower:
            head.append(dic_voca_lower[tkn])
        else:
            head.append(1)
            
    heads.append(head)
    
    body = []
    for tkn in row[2].lower().strip().split():
        if tkn in dic_voca_lower:
            body.append(dic_voca_lower[tkn])
        else:
            body.append(1)
            
    bodys.append(body)
    
print n+1, 'Done'
print datetime.datetime.now().isoformat() # ~5 mins

2020-05-24T09:06:07.398210
10000 20000 30000 40000 50000 60000 70000 80000 87530 Done
2020-05-24T09:06:19.948299


In [96]:
print datetime.datetime.now().isoformat()
[np_heads, np_bodys] = fit_length([heads, bodys], 25, 2100)
print datetime.datetime.now().isoformat() # ~3 mins

2020-05-24T09:06:19.956499
2020-05-24T09:06:32.539512


In [97]:
print datetime.datetime.now().isoformat()
t_trainpath = path_processed_data + '/rw-test/test_title.npy'
np.save(t_trainpath, np_heads)
b_trainpath = path_processed_data + '/rw-test/test_body.npy'
np.save(b_trainpath, np_bodys)
l_trainpath = path_processed_data + '/rw-test/test_label.npy'
np.save(l_trainpath, labels)
print datetime.datetime.now().isoformat()

2020-05-24T09:06:32.548806
2020-05-24T09:06:32.968777


In [101]:
with open('../data/real_world_articles/old/whole/rw-test/test_label.npy', 'r') as f:
    tt = np.load(f)
tt.shape

(87530,)